# Regression Model B: Field 5

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 5 (F5)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E5" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E5" ),]
dE4=d4[which(d4$Estate=="E5" ),]
dE5=d5[which(d5$Estate=="E5" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
10,1,1,1,1,1,1,1,1,1,1,1,0
14,1,1,1,1,1,1,1,1,1,0,0,2
6,1,1,1,1,1,1,1,0,0,1,1,2
2,1,1,1,1,1,1,1,1,0,0,0,3
6,1,1,1,1,1,1,1,0,0,0,0,4
4,1,1,1,1,1,0,0,1,1,0,0,4
2,1,1,1,1,1,0,0,0,0,0,0,6
,0,0,0,0,0,6,6,14,16,28,28,98


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            28            28            14            16 
      pd.time       pd.cost 
            6             6 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper",col=brewer.pal(n=8, name="RdYlBu"))


## Regression

Fit the data using multiple regression for all variables. 

In [10]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=df2)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = df2)

Residuals:
   Min     1Q Median     3Q    Max 
-55.35 -14.87  -0.23  20.15  60.81 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)    9.390e+01  3.870e+01   2.426   0.0205 *
rainfall       2.182e-02  4.761e-02   0.458   0.6496  
foliar        -2.516e+00  1.752e+00  -1.437   0.1597  
prun.time      1.535e+01  1.122e+01   1.368   0.1801  
prun.cost      1.699e-03  4.493e-03   0.378   0.7076  
manuring.time  1.412e+01  8.568e+00   1.648   0.1083  
manuring.cost -2.510e-04  3.135e-04  -0.801   0.4288  
pd.time       -1.184e+01  9.267e+00  -1.277   0.2100  
pd.cost        4.424e-04  5.134e-03   0.086   0.9318  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 30.66 on 35 degrees of freedom
Multiple R-squared:  0.3798,	Adjusted R-squared:  0.2381 
F-statistic: 2.679 on 8 and 3

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [11]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,5.348645e-01,5.348645e-01,5.689963e-04,0.981104832
foliar,1,1.128592e+04,1.128592e+04,1.200612e+01,0.001419906
prun.time,1,2.864873e+03,2.864873e+03,3.047692e+00,0.089625943
prun.cost,1,6.193081e+01,6.193081e+01,6.588286e-02,0.798931312
manuring.time,1,2.089889e+03,2.089889e+03,2.223253e+00,0.144904830
manuring.cost,1,1.286936e+03,1.286936e+03,1.369060e+00,0.249879482
pd.time,1,2.552309e+03,2.552309e+03,2.715182e+00,0.108347606
pd.cost,1,6.981084e+00,6.981084e+00,7.426575e-03,0.931816379
Residuals,35,3.290049e+04,9.400140e+02,NA,NA


### Using stepwise

In [12]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=309.15
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- pd.cost        1      6.98 32907 307.16
- prun.cost      1    134.38 33035 307.33
- rainfall       1    197.41 33098 307.41
- manuring.cost  1    602.53 33503 307.95
<none>                       32900 309.15
- pd.time        1   1533.17 34434 309.15
- prun.time      1   1758.97 34659 309.44
- foliar         1   1940.43 34841 309.67
- manuring.time  1   2552.46 35453 310.44

Step:  AIC=307.16
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time

                Df Sum of Sq   RSS    AIC
- prun.cost      1    133.14 33041 305.34
- rainfall       1    191.07 33099 305.41
- manuring.cost  1    662.28 33570 306.04
<none>                       32907 307.16
- foliar         1   1937.15 34845 307.68
- prun.time      1   2164.04 35072 307.96
- pd.time        1   2552.31 35460 308.45



Call:
lm(formula = yield ~ foliar + prun.time + manuring.time + pd.time, 
    data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-55.251 -17.058  -0.479  24.788  62.712 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    109.787     28.033   3.916 0.000351 ***
foliar          -3.084      1.466  -2.104 0.041866 *  
prun.time       13.659      8.434   1.619 0.113412    
manuring.time   10.240      6.252   1.638 0.109482    
pd.time        -11.552      6.364  -1.815 0.077194 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 29.57 on 39 degrees of freedom
Multiple R-squared:  0.3574,	Adjusted R-squared:  0.2914 
F-statistic: 5.422 on 4 and 39 DF,  p-value: 0.001435


       foliar     prun.time manuring.time       pd.time 
     1.221979      1.198490      1.002974      1.073111 



Call:
lm(formula = yield ~ foliar + prun.time + manuring.time + pd.time, 
    data = df2)

Residuals:
    Min      1Q  Median      3Q     Max 
-55.251 -17.058  -0.479  24.788  62.712 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    109.787     28.033   3.916 0.000351 ***
foliar          -3.084      1.466  -2.104 0.041866 *  
prun.time       13.659      8.434   1.619 0.113412    
manuring.time   10.240      6.252   1.638 0.109482    
pd.time        -11.552      6.364  -1.815 0.077194 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 29.57 on 39 degrees of freedom
Multiple R-squared:  0.3574,	Adjusted R-squared:  0.2914 
F-statistic: 5.422 on 4 and 39 DF,  p-value: 0.001435


## Prediction

Predict with the resample values.

In [13]:
set.seed(123)
df3=sample(1:nrow(df2),12)
df4=df2[df3,]
estimate=predict(selectedMod,newdata=df4)
estimate
sum.estimate=sum(estimate)
sum.estimate

df5=data.frame(estimate,sum.estimate)

13        34        18        37        38         2        21        43 
172.03814  82.82333 144.27058 135.79250 131.39592 153.18845 111.11895 137.76749 
       20        16        33        15 
102.65069 114.98746 108.49418 120.17817

[1] 1514.706